# Load Library


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import KNNImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, TimeSeriesSplit, train_test_split

# Init data


In [4]:
# train_df = pd.read_csv("../datasets/train.csv")
train_df = pd.read_csv(
    '../datasets/train.csv',
    usecols=['date', 'num_sold'],
    skiprows=lambda x: x > 0 and np.random.rand() > 0.1
)
train_df['date'] = pd.to_datetime(train_df['date'])

# Transforming & Forecasting data


- Membuat fitur lag dan fitur terkait tanggal untuk supervised learning.


In [6]:
def create_features(data, lags=3, date_features=None, dropna=True, fill_value=None):
    """
    Membuat fitur lag dan fitur terkait tanggal untuk supervised learning.

    Args:
        data (pd.DataFrame): DataFrame input dengan kolom 'ds' (datetime) dan 'y' (target).
        lags (int): Jumlah lag yang akan dibuat. Defaultnya 3.
        date_features (list, optional): List fitur tanggal yang akan dibuat. 
            Pilihan: 'year', 'quarter', 'month', 'dayofweek', 'dayofyear', 'weekofyear'.
            Jika None, hanya 'month' dan 'dayofweek' yang dibuat.
        dropna (bool, optional): Apakah akan menghapus baris dengan NaN (karena lag). Defaultnya True.
        fill_value (any, optional): Nilai yang akan digunakan untuk mengisi NaN jika dropna=False. Defaultnya None.

    Returns:
        pd.DataFrame: DataFrame dengan fitur-fitur baru.
        int: Jumlah baris yang dihapus jika dropna=True, atau 0 jika dropna=False.
        Mengembalikan None jika input tidak valid.

    Raises:
        TypeError: Jika input data bukan DataFrame atau kolom 'ds' bukan datetime.
        ValueError: Jika lags kurang dari 1.
    """

    if not isinstance(data, pd.DataFrame):
        raise TypeError("Input data harus berupa DataFrame.")
    if not pd.api.types.is_datetime64_any_dtype(data['ds']):
        raise TypeError("Kolom 'ds' harus bertipe datetime.")
    if lags < 1:
        raise ValueError("Jumlah lag harus minimal 1.")

    lagged_data = {f"lag_{i}": data['y'].shift(i) for i in range(1, lags + 1)}
    lagged_df = pd.DataFrame(lagged_data)

    if date_features is None:
        date_features = ['month', 'dayofweek']

    date_feature_data = {}
    for feature in date_features:
        try:
            date_feature_data[feature] = getattr(data['ds'].dt, feature)
        except AttributeError:
            print(f"Fitur tanggal {feature} tidak dikenal atau bukan atribut datetime. Melewati.")

    date_features_df = pd.DataFrame(date_feature_data)

    result = pd.concat([data, lagged_df, date_features_df], axis=1)

    rows_dropped = 0
    if dropna:
        rows_dropped = len(result)
        result.dropna(inplace=True)
        rows_dropped -= len(result)
    elif fill_value is not None:
        result.fillna(fill_value, inplace=True)

    return result, rows_dropped

- Load & prepare new data


In [8]:
train_df.rename(columns={"date": "ds", "num_sold": "y"}, inplace=True)
train_df['ds'] = pd.to_datetime(train_df['ds'])

# Unpacking tuple: data_df akan berisi DataFrame, dropped akan berisi jumlah baris yang dihapus
data, dropped = create_features(train_df)

# Sekarang Anda bisa memanggil head() pada DataFrame
print(f"Jumlah baris yang dihapus: {dropped}")

Jumlah baris yang dihapus: 3


- Cari hyperparameter terbaik menggunakan grid search dan random search


In [ ]:
X = data.drop(columns=["y", "ds"])
y = data["y"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# TimeSeriesSplit untuk time series data
tscv = TimeSeriesSplit(n_splits=5) # 5 fold cross validation

# Definisikan parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
}

# Inisialisasi model
model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Grid Search
grid_search = GridSearchCV(model, param_grid, cv=tscv, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1) # n_jobs=-1 menggunakan semua core CPU
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best score:", np.sqrt(-grid_search.best_score_))

# Random Search (Jika rentang parameter sangat luas)
n_iter_search = 20 # Jumlah iterasi random search
random_search = RandomizedSearchCV(model, param_grid, n_iter=n_iter_search, cv=tscv, scoring='neg_mean_squared_error', verbose=1, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

print("\nRandom Search")
print("Best parameters:", random_search.best_params_)
print("Best score:", np.sqrt(-random_search.best_score_))

# Evaluasi model terbaik pada data test
best_model = grid_search.best_estimator_ # atau random_search.best_estimator_
predictions = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"RMSE pada data test dengan parameter terbaik: {rmse:.2f}")

- Train Model


In [ ]:
model = XGBRegressor(n_estimators=50, learning_rate=0.01, max_depth=3, random_state=42)
model.fit(X_train, y_train)

- Hitung score RMSE, MAE, MAPE, dan hitung naive prediction


In [ ]:
y_true = y_test.values
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

# Naive forecast
naive_pred = y_test.shift(1)  # Menggunakan nilai sebelumnya sebagai prediksi
naive_mae = mean_absolute_error(y_test[1:], naive_pred[1:])
naive_rmse = np.sqrt(mean_squared_error(y_test[1:], naive_pred[1:]))

print(f"Naive Forecast MAE: {naive_mae:.2f}")
print(f"Naive Forecast RMSE: {naive_rmse:.2f}")

Prediksi sales untuk satu tahun ke depan


In [ ]:
future_steps = 365
lags = 5  # Sesuaikan dengan jumlah lag saat melatih model
last_known_data = data.iloc[-lags:].copy()  # Gunakan sejumlah baris terakhir sesuai jumlah lag
future_forecast = []

for _ in range(future_steps):
    # Ambil fitur lagging terakhir
    if len(last_known_data) < lags:
        raise ValueError(f"Insufficient data for lag features. Expected: {lags}, got: {len(last_known_data)}")
    
    future_input = last_known_data["y"].values[-lags:].reshape(1, -1)
    
    # Prediksi nilai berikutnya
    pred = model.predict(future_input)[0]
    future_forecast.append(pred)
    
    # Perbarui data lag
    last_known_data = pd.concat(
        [last_known_data.iloc[1:], pd.DataFrame({"y": [pred]})], ignore_index=True
    )

# Data Visualization


- Basic


In [ ]:
train_data = train_df.copy()
train_data["Type"] = "Train Data"

forecast_data = pd.DataFrame({
    "ds": pd.date_range(start=train_data["ds"].iloc[-1], periods=future_steps, freq="D"),
    "y": future_forecast,
    "Type": "Forecast",
})

# Gabungkan data historis dan prediksi untuk visualisasi
combined_data = pd.concat([train_data, forecast_data], ignore_index=True)

# Plot Data Historis dan Prediksi
plt.figure(figsize=(14, 8))
sns.lineplot(data=combined_data, x="ds", y="y", hue="Type", palette=["blue", "red"], linewidth=1.5)

# Judul dan Label
plt.title("Visualisasi Data Historis dan Prediksi", fontsize=16)
plt.xlabel("Tanggal", fontsize=12)
plt.ylabel("Jumlah Terjual", fontsize=12)
plt.legend(title="Tipe Data")
plt.grid(alpha=0.3)
plt.show()


- With confidence interval


In [ ]:

# Data Historis dan Prediksi dengan Interval Kepercayaan
forecast_data = pd.DataFrame({
    "ds": pd.date_range(start=data["ds"].iloc[-1], periods=future_steps, freq="D"),
    "y": future_forecast,
    "yhat_lower": np.array(future_forecast) * 0.9,  # Contoh interval bawah (90% dari nilai)
    "yhat_upper": np.array(future_forecast) * 1.1,  # Contoh interval atas (110% dari nilai)
    "Type": "Forecast",
})

# Gabungkan data historis dan prediksi
combined_data = pd.concat([data, forecast_data], ignore_index=True)

# Plot Data Historis dan Prediksi
plt.figure(figsize=(15, 6))
sns.lineplot(data=combined_data, x="ds", y="y", hue="Type", palette="Blues_d", linewidth=1.5)

# Tambahkan Area Confidence Interval
plt.fill_between(
    forecast_data["ds"],
    forecast_data["yhat_lower"],
    forecast_data["yhat_upper"],
    color="red",
    alpha=0.3,
    label="Confidence Interval",
)

# Judul dan Label
plt.title("Visualisasi Data Historis dan Prediksi dengan Interval Kepercayaan", fontsize=16)
plt.xlabel("Tanggal", fontsize=12)
plt.ylabel("Jumlah Terjual", fontsize=12)
plt.legend(title="Tipe Data")
plt.grid(alpha=0.3)
plt.show()


- Residuals


In [ ]:
# Visualisasi Residuals
residuals = y_test - y_pred
plt.figure(figsize=(12, 6))
plt.plot(residuals, label="Residuals", color="orange")
plt.axhline(0, linestyle="--", color="red")
plt.title("Residuals Plot")
plt.xlabel("Index")
plt.ylabel("Error")
plt.legend()
plt.grid(alpha=0.3)
plt.show()